# hypothesis testing

a nice set of working things

In [ ]:
#chi squared
from scipy.stats import chi2_contingency, chi2

def chi_test(arr):

  """
  chi_test function takes a pandas df and breaks it into a numpy array for testing of independence
  can also do fisher exact if testing on small counts...
  
  input:
  arr - numpy array of count data
  """

  alpha = 0.05

  #array is struct of (pna, non-pna)
  stat, p, dof, expected = chi2_contingency(arr, correction = False)

  prob = 1 - alpha

  # interpet test statistic
  critical = chi2.ppf(prob, dof)
  print("probability ={}, critical ={}, stat={}, p value = {}".format(prob, critical, stat, p))

  if abs(stat) >= critical:
    print('accept alternative hypothesis; counts are dependent on categories')
  else:
    print('maintain null hypothesis; counts are not dependent on categories')

  print('contingency table:\n', arr)

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

def z_test(yes_count, yes_target, yes_label, no_count, no_target, no_label, alpha = 0.05):
  
  """
  z test function runs a z test to see if one proportion is higher with any significance
  yes proportion must be smaller than no, other wise the alternative in the stats models function needs to be altered
  also gives effect sizes
  
  inputs:
  yes_count = sum count of observations for one class
  yes_target = sum count of successful observations for one class
  no_count = sum count of observations for one class
  no_target = sum count of successful observations for one class
  
  """
  
  if yes_target/yes_count > no_target/no_count:
    print('proportions are not aligned properly')
    return None

  nobs = np.array([yes_count, no_count])
  count = np.array([yes_target, no_target])
  print("observations" , nobs)
  print("counts", count)
  
  print("proportion ", yes_label, ": ", round(yes_target/yes_count,2))
  print("proportion ", no_label, ": ", round(no_target/no_count,2))

  stat, pval = proportions_ztest(count, nobs, alternative = 'smaller')
  #smaller is when elements 0 have smaller proportion than elements 1

  print('{0:0.3f}'.format(pval))

  print(pval)  

  if pval <= alpha:
    print("accept alternative hypothesis that ", no_label, " patients have a higher pick up rate")
  else:
    print("maintain null hypothesis that there is no statistical significance that ", no_label, " patients have a higher pick up rate")
    
  #put smaller proportion thru last to get a positive effect size
  effect = proportion_effectsize(no_target/no_count, yes_target/yes_count)
  print("proportion effect size: ", effect)

In [ ]:

#shapiro-wilks normality test
from scipy.stats import shapiro

def normality_test(data, alpha = 0.05):
  stat, p = shapiro(data)
  print('Statistics=%.3f, p=%.3f' % (stat, p))
  
  response = 1 #0 = normal
  
  if p >= alpha:
    print("data is normally distributed")
  else:
    print("data is not normally distributed")
    response = 0
  
  return response

In [ ]:
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.stats import tiecorrect, rankdata, norm

def mann_whitney(data1, data2, alpha = .05): 
  #calculates significance of diff in means and effect
  #this is a base two sided test
  #data 1 must be less than data 2
  #otherwise tweak alternative

  # compare samples
  stat, p = mannwhitneyu(data1, data2, alternative="less")
  print('Statistics=%.3f, p=%.3f' % (stat, p))
  # interpret
  #alpha = 0.05
  if p > alpha:
    print('Same distribution (fail to reject H0)', '\nIn terms of this analysis this means that the difference in means is NOT due to the different categories!')
  else:
    print('Different distribution (reject H0)', '\nIn terms of this analysis this means that the difference in means is due to the different categories!')

  #calculate effect size:
  #borroewd from https://github.com/Hatchin/Mann-Whitney-U-Test/blob/master/mannwhitney.py
  n1 = len(data1)
  n2 = len(data2)
  ranked = rankdata(np.concatenate((data1, data2)))
  rankx = ranked[0:n1]  # get the x-ranks
  u1 = n1*n2 + (n1*(n1+1))/2.0 - np.sum(rankx, axis=0)  # calc U for x
  u2 = n1*n2 - u1  # remainder is U for y

  # use the min(u1, u2) as u-stat
  if u1 <= u2:
      stat_a, larger = u1, 1
  else:
      stat_a, larger = u2, 2

  # compute the effect size    
  effect = 1 - (2*stat_a)/(n1*n2) 
  
  if effect < .3:
    print('This has a small effect; the effect size for this test is : %.3f' % effect)
  elif effect > .5:
    print('This has a large effect; the effect size for this test is : %.3f' % effect)
  else:
    print('This has a medium effect; the effect size for this test is : %.3f' % effect)

In [ ]:
from scipy.stats import ttest_ind
import numpy as np

def t_test(data1, data2, alpha = 0.05):
  """
  t test function to measure if mean of data1 is less than mean of data2
  """
  d1_var = np.var(data1)
  d2_var = np.var(data2)
  
  equal_var = False
  
  if round(d1_var,4) == round(d2_var,4):
    equal_var = True
  
  stat, p = ttest_ind(data1, data2, axis = 0, equal_var = equal_var, alternative = 'less')
  
  print('Statistics=%.3f, p=%.3f' % (stat, p))
  
  if p > alpha:
    print("maintain null hypothesis that means are not significantly different")
  else:
    print("accept alternative hypothesis that means are significantly different")
    
  #effect size (cohen's d calc)
  #machine learning mastery formula
  n1, n2 = len(data1), len(data2)
  
  u1, u2 = np.mean(data1), np.mean(data2)
  
  #pooled standard deviation
  s = sqrt((((n1 -1) * d1_var) + ((n2 - 1) * d2_var))/(n1 + n2 -2))
  
  #flipped u so as to show a positive effect measure
  d = (u2 - u1)/s
           
  if d < .2: 
    print('Cohens d shows a small effect (if your data is nor normal/gaussian this is junk!): %.3f' % d)
  elif d > .8:
    print('Cohens d shows a large effect (if your data is nor normal/gaussian this is junk!): %.3f' % d)
  else:
    print('Cohens d shows a medium effect (if your data is nor normal/gaussian this is junk!): %.3f' % d)